# Assignment 5, Question 6: Data Transformation

**Points: 20**

Transform and engineer features from the clinical trial dataset.

## Setup

In [373]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import utilities
from q3_data_utils import load_data, clean_data, transform_types, create_bins, fill_missing

df = load_data('output/q5_cleaned_data.csv')
print(f"Loaded {len(df)} patients")

# Prewritten visualization functions for transformation analysis
def plot_distribution(series, title, figsize=(10, 6)):
    """
    Create a histogram of a numeric series.
    
    Args:
        series: pandas Series with numeric data
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.hist(bins=30)
    plt.title(title)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

def plot_value_counts(series, title, figsize=(10, 6)):
    """
    Create a bar chart of value counts.
    
    Args:
        series: pandas Series with value counts
        title: Chart title
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    series.plot(kind='bar')
    plt.title(title)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

Loaded 10000 patients


## Part 1: Type Conversions (5 points)

1. Convert 'enrollment_date' to datetime using the `transform_types()` utility
2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
3. Ensure all numeric columns are proper numeric types
4. Display the updated dtypes

In [374]:
# TODO: Type conversions
# 1. Use transform_types() to convert enrollment_date to datetime
# 2. Convert categorical columns ('site', 'intervention_group', 'sex') to category dtype
# 3. Ensure all numeric columns are proper numeric types
# 4. Display the updated dtypes using df.dtypes
df = transform_types(df, {'enrollment_date': 'datetime'})
df = transform_types(df, {'site': 'category', 'intervention_group': 'category', 'sex': 'category'})

df = transform_types(df, {"age": "numeric", "bmi": "numeric", "systolic_bp": "numeric", "diastolic_bp": "numeric", "cholesterol_total": "numeric", "cholesterol_hdl": "numeric", "cholesterol_ldl": "numeric", "glucose_fasting": "numeric", "follow_up_months": "numeric", "adherence_pct": "numeric"})
print(df.dtypes)

patient_id                    object
age                            int64
sex                         category
bmi                          float64
enrollment_date       datetime64[ns]
systolic_bp                  float64
diastolic_bp                 float64
cholesterol_total            float64
cholesterol_hdl              float64
cholesterol_ldl              float64
glucose_fasting              float64
site                        category
intervention_group          category
follow_up_months               int64
adverse_events                 int64
outcome_cvd                   object
adherence_pct                float64
dropout                       object
dtype: object


## Part 2: Feature Engineering (8 points)

Create these new calculated columns:

1. `cholesterol_ratio` = cholesterol_ldl / cholesterol_hdl
2. `bp_category` = categorize systolic BP:
   - 'Normal': < 120
   - 'Elevated': 120-129
   - 'High': >= 130
3. `age_group` using `create_bins()` utility:
   - Bins: [0, 40, 55, 70, 100]
   - Labels: ['<40', '40-54', '55-69', '70+']
4. `bmi_category` using standard BMI categories:
   - Underweight: <18.5
   - Normal: 18.5-24.9
   - Overweight: 25-29.9
   - Obese: >=30

In [375]:
# TODO: Calculate cholesterol ratio
df['cholesterol_ratio'] = df['cholesterol_total'] / df['cholesterol_hdl']

In [376]:
# TODO: Categorize blood pressure
df['bp_category'] = pd.cut(df['systolic_bp'], bins=[-float('inf'), 120, 129, float('inf')], labels=['Normal', 'Elevated', 'High'])

**Note:** The `create_bins()` function has an optional `new_column` parameter. If you don't specify it, the new column will be named `{original_column}_binned`. You can use `new_column='age_group'` to give it a custom name.


In [377]:
# TODO: Create age groups
create_bins(df, 'age', bins=[0, 40, 55, 70, 100], labels=['<40', '40-54', '55-69', '70+'], new_column='age_group')

,patient_id,age,sex,bmi,enrollment_date,systolic_bp,diastolic_bp,cholesterol_total,cholesterol_hdl,cholesterol_ldl,glucose_fasting,site,intervention_group,follow_up_months,adverse_events,outcome_cvd,adherence_pct,dropout,cholesterol_ratio,bp_category,age_group
0,P00001,80,F,29.3,2022-05-01,123.0,80.0,120.0,55.0,41.0,118.0,site b,control,20,0,No,24.0,No,2.181818,Elevated,70+
1,P00002,80,Female,26.0,2022-01-06,139.0,81.0,206.0,58.0,107.0,79.0,site a,control,24,0,No,77.0,No,3.551724,High,70+
2,P00003,82,Female,-1.0,2023-11-04,123.0,86.0,172.0,56.0,82.0,77.0,site c,treatment b,2,0,Yes,70.0,No,3.071429,Elevated,70+
3,P00004,95,Female,25.4,2022-08-15,116.0,77.0,200.0,56.0,104.0,115.0,site d,treatment b,17,0,No,62.0,No,3.571429,Normal,70+
4,P00005,95,M,26.0,2023-04-17,97.0,71.0,185.0,78.0,75.0,113.0,site e,treatment a,9,0,yes,62.0,Yes,2.371795,Normal,70+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,P09996,72,Male,23.2,2022-04-11,122.0,73.0,182.0,54.0,92.0,97.0,site c,treatment b,21,0,No,20.0,No,3.370370,Elevated,70+
9996,P09997,100,Female,28.9,2023-02-10,124.0,78.0,157.0,56.0,70.0,102.0,site c,control,11,0,No,57.0,No,2.803571,Elevated,70+
9997,P09998,78,F,23.8,2023-11-05,110.0,63.0,154.0,69.0,71.0,114.0,site c,treatment a,2,1,No,77.0,No,2.231884,Normal,70+
9998,P09999,86,F,27.0,2022-08-27,139.0,98.0,196.0,38.0,119.0,126.0,site a,control,16,0,no,63.0,No,5.157895,High,70+


In [378]:
# TODO: Create BMI categories
df['bmi_category'] = pd.cut(df['bmi'], bins=[-float('inf'), 18.5, 25, 29.9, float('inf')], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

## Part 3: String Cleaning (2 points)

If there are any string columns that need cleaning:
1. Convert to lowercase
2. Strip whitespace
3. Replace any placeholder values

In [379]:
# TODO: String cleaning
df['sex'] = df['sex'].str.strip().str.lower()
df['sex'] = df['sex'].replace({"f": "female", "m": "male"})
df["outcome_cvd"] = df["outcome_cvd"].str.strip().str.lower()
df.to_csv("output/q6_load_this_data.csv", index = False)

## Part 4: One-Hot Encoding (5 points)

Create dummy variables for categorical columns:
1. One-hot encode 'intervention_group' using `pd.get_dummies()`
2. One-hot encode 'site'
3. Drop the original categorical columns
4. Show the new shape and column names

In [380]:
# TODO: One-hot encoding
df = pd.get_dummies(df, columns=['intervention_group', 'site'], dtype='int64')
print(df.shape)
print(df.columns[-8:])  # Display new one-hot encoded columns

(10000, 27)
Index(['intervention_group_control', 'intervention_group_treatment a',
       'intervention_group_treatment b', 'site_site a', 'site_site b',
       'site_site c', 'site_site d', 'site_site e'],
      dtype='object')


## Part 5: Save Transformed Data

Save the fully transformed dataset to `output/q6_transformed_data.csv`

In [381]:
# TODO: Save transformed data
# df_transformed.to_csv('output/q6_transformed_data.csv', index=False)
df.to_csv('output/q6_transformed_data.csv', index=False)